In [2]:
import time

import numpy as np
import pandas as pd
import sklearn.datasets, sklearn.metrics, sklearn.model_selection, sklearn.tree

import subprocess, sys

In [10]:
!cd ../ && make print-SOURCES

SOURCES = cpp/criterion.cpp cpp/factories.cpp cpp/gbt.cpp cpp/lltrees.cpp cpp/metrics.cpp cpp/node.cpp cpp/tree.cpp cpp/wrapper.cpp


In [132]:
# !cd ../ && make clean
!cd ../ && make
subprocess.call([sys.executable, "lltrees_python_import_for_debug.py"])

g++ -g -o build/tree.o cpp/tree.cpp -I/usr/include/python3.10 -I/usr/include/python3.10  -Wno-unused-result -Wsign-compare -g      -fstack-protector-strong -Wformat -Werror=format-security  -DNDEBUG -g -fwrapv -O2 -Wall -fPIC -c -std=c++20 -DBOOST_BIND_GLOBAL_PLACEHOLDERS -DBOOST_ALLOW_DEPRECATED_HEADERS
g++ -g -o build/lltrees.so build/criterion.o build/factories.o build/gbt.o build/lltrees.o build/metrics.o build/node.o build/tree.o build/wrapper.o -L/usr/lib/python3.10/config-3.10-x86_64-linux-gnu -L/usr/lib/x86_64-linux-gnu  -lcrypt -ldl  -lm -lm  -shared -Wl,-rpath,/usr/lib  -lpython3.10 -lboost_python310 -lboost_numpy310
-----------------------------------------
mode :              regression
epochs :            1
learning_rate :     0.1
metric :            mae
criterion :         absolute_error
max_depth :         3
min_leaf_size :     1
verbose :           1
-----------------------------------------
Type of Training Data : float64
Configuration mode : regression
Epoch : 1     M

Traceback (most recent call last):
  File "/home/alexandre/Desktop/lltrees/demo/lltrees_python_import_for_debug.py", line 46, in <module>
    print("rmse: %.2f" % np.sqrt(sklearn.metrics.mean_squared_error(Y_test,YP)))
  File "/home/alexandre/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py", line 442, in mean_squared_error
    y_type, y_true, y_pred, multioutput = _check_reg_targets(
  File "/home/alexandre/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py", line 102, in _check_reg_targets
    y_pred = check_array(y_pred, ensure_2d=False, dtype=dtype)
  File "/home/alexandre/.local/lib/python3.10/site-packages/sklearn/utils/validation.py", line 899, in check_array
    _assert_all_finite(
  File "/home/alexandre/.local/lib/python3.10/site-packages/sklearn/utils/validation.py", line 146, in _assert_all_finite
    raise ValueError(msg_err)
ValueError: Input contains NaN.


1

# make_regression

In [39]:
X, Y = sklearn.datasets.make_regression(n_samples=1000, n_features=8, n_informative=5, n_targets=1, noise=1, random_state=42)
X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(X, Y, test_size=0.3, random_state=42)

In [ ]:
sys.path.append('/home/alexandre/Desktop/lltrees/build')
import lltrees
conf ={
    'epochs' : 1,
    'learning_rate' : 1,
    'metric' : 'mae', # Possible metrics for the moment : mae, mse
    'max_depth' : 5,
    'min_leaf_size' : 2,
    'criterion' : "absolute_error",  # Possible metrics for the moment : variance, absolute_error
    'verbose' : 1, 
}
my_lltree = lltrees.lltree()
my_lltree.set_conf(conf)
my_lltree.get_conf()

start_time = time.time()
my_lltree.fit(X_train, Y_train, X_test, Y_test)
print("FIT --- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
YP = my_lltree.predict(X_test)
print("PREDICT --- %s seconds ---" % (time.time() - start_time))

print("rmse: %.2f" % np.sqrt(sklearn.metrics.mean_squared_error(Y_test,YP)))
print("mae: %.2f" % sklearn.metrics.mean_absolute_error(Y_test,YP))
print("r2: %.2f" % sklearn.metrics.r2_score(Y_test,YP))

In [14]:
my_lltree = sklearn.tree.DecisionTreeRegressor(max_depth = 5, criterion ="absolute_error", random_state = 0)

start_time = time.time()
my_lltree.fit(X_train, Y_train)
print("FIT --- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
YP = my_lltree.predict(X_test)
print("PREDICT --- %s seconds ---" % (time.time() - start_time))

print(YP[0:10])
print("rmse: %.2f" % np.sqrt(sklearn.metrics.mean_squared_error(Y_test,YP)))
print("mae: %.2f" % sklearn.metrics.mean_absolute_error(Y_test,YP))
print("r2: %.2f" % sklearn.metrics.r2_score(Y_test,YP))

FIT --- 0.020457029342651367 seconds ---
PREDICT --- 0.00027060508728027344 seconds ---
[  -5.25635722  127.29805997 -110.4560012   122.24421396 -110.4560012
  -39.90414254  106.73581898  -39.90414254  223.68925908  240.37517626]
rmse: 63.68
mae: 49.35
r2: 0.74


In [ ]:
conf ={
    'epochs' : 50,
    'learning_rate' : 0.1,
    'metric' : 'mae', # Possible metrics for the moment : mae, mse
    'max_depth' : 5,
    'min_leaf_size' : 2,
    'criterion' : "absolute_error",  # Possible metrics for the moment : variance, absolute_error
    'verbose' : 0, 
}
my_lltree = lltrees.lltree()
my_lltree.set_conf(conf)
my_lltree.get_conf()

start_time = time.time()
my_lltree.fit(X_train, Y_train, X_test, Y_test)
print("FIT --- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
YP = my_lltree.predict(X_test)
print("PREDICT --- %s seconds ---" % (time.time() - start_time))

print("rmse: %.2f" % np.sqrt(sklearn.metrics.mean_squared_error(Y_test,YP)))
print("mae: %.2f" % sklearn.metrics.mean_absolute_error(Y_test,YP))
print("r2: %.2f" % sklearn.metrics.r2_score(Y_test,YP))

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
my_lltree = GradientBoostingRegressor(learning_rate = 0.1, n_estimators =50, max_depth = 5,
                                 loss = 'absolute_error', criterion = 'squared_error',
                                 random_state = 0)

start_time = time.time()
my_lltree.fit(X_train, Y_train)
print("FIT --- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
YP = my_lltree.predict(X_test)
print("PREDICT --- %s seconds ---" % (time.time() - start_time))

print("rmse: %.2f" % np.sqrt(sklearn.metrics.mean_squared_error(Y_test,YP)))
print("mae: %.2f" % sklearn.metrics.mean_absolute_error(Y_test,YP))
print("r2: %.2f" % sklearn.metrics.r2_score(Y_test,YP))

# make_classification

In [ ]:
# X, Y = sklearn.datasets.make_classification(n_samples=1000, n_features=8, n_informative=5, n_classes=2, random_state=42)
# X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(X, Y, test_size=0.3, random_state=42)

In [ ]:
# conf ={
#     'epochs' : 50,
#     'learning_rate' : 0.1,
#     'metric_name' : 'mae', # Possible metrics for the moment : mae, mse
#     'lltree_max_depth' : 3,
#     'lltree_min_size_split' : 1,
#     'lltree_criterion' : "gini",  # Possible metrics for the moment : variance, absolute_error
#     'verbose' : 1,  # Possible metrics for the moment : variance, absolute_error
# }
# my_lltree = lltrees.lltree()
# my_lltree.set_conf(conf)
# my_lltree.get_conf()

# start_time = time.time()
# my_lltree.fit(X_train, Y_train)
# print("FIT --- %s seconds ---" % (time.time() - start_time))

# start_time = time.time()
# YP = my_lltree.predict(X_test)
# print("PREDICT --- %s seconds ---" % (time.time() - start_time))

# print("accuracy_score: %.2f" % np.sqrt(sklearn.metrics.accuracy_score(Y_test,YP)))
# print("log_loss: %.2f" % sklearn.metrics.log_loss(Y_test,YP))
# print("f1_score: %.2f" % sklearn.metrics.f1_score(Y_test,YP))